In [ ]:
'''
A Convolutional Network implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

In [4]:
import numpy as np
import tensorflow as tf
import read_mnist_data as rmd

# read dataset from disk
DATA_FOLDER = '/home/chengming/TensorFlow-Examples/data/'
TRAIN_IMAGES_SET = 'train-images-idx3-ubyte.gz'
TRAIN_LABELS_SET = 'train-labels-idx1-ubyte.gz'
TEST_IMAGES_SET = 't10k-images-idx3-ubyte.gz'
TEST_LABELS_SET = 't10k-labels-idx1-ubyte.gz'
validation_size = 5000
with open(DATA_FOLDER + TRAIN_IMAGES_SET, 'rb') as f:
    train_images = rmd.extract_images(f)
with open(DATA_FOLDER + TRAIN_LABELS_SET, 'rb') as f:
    train_labels = rmd.extract_labels(f, one_hot = True)
with open(DATA_FOLDER + TEST_IMAGES_SET, 'rb') as f:
    test_images = rmd.extract_images(f)
with open(DATA_FOLDER + TEST_LABELS_SET, 'rb') as f:
    test_labels = rmd.extract_labels(f, one_hot = True)
#validation_images = train_images[:validation_size]
#validation_labels = train_labels[:validation_size]
#train_images = train_images[validation_size:]
#train_labels = train_labels[validation_size:]

train_shape = train_images.shape
test_shape = test_images.shape
train_images = train_images.reshape(train_shape[0], train_shape[1]*train_shape[2])
test_images = test_images.reshape(test_shape[0], test_shape[1]*test_shape[2])


print train_images.shape
print train_labels.shape, '\t', train_labels[0][0:6], train_labels[0][6:]
print test_images.shape
print test_labels.shape, '\t', test_labels[0]

('Extracting', '/home/chengming/TensorFlow-Examples/data/train-images-idx3-ubyte.gz')
('Extracting', '/home/chengming/TensorFlow-Examples/data/train-labels-idx1-ubyte.gz')
('Extracting', '/home/chengming/TensorFlow-Examples/data/t10k-images-idx3-ubyte.gz')
('Extracting', '/home/chengming/TensorFlow-Examples/data/t10k-labels-idx1-ubyte.gz')
(60000, 784)
(60000, 10) 	[ 0.  0.  0.  0.  0.  1.] [ 0.  0.  0.  0.]
(10000, 784)
(10000, 10) 	[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]


In [5]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

# Network Parameters
n_input = train_images.shape[1] # MNIST data input (img shape: 28*28)
n_classes = train_labels.shape[1] # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input, dropout probability needs to be a tensor
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [6]:
# Create some wrappers for simplicity

# conv2d
# input shape = [batch, in_height, in_width, in_channels]
# filter shape = [filter_height, filter_width, in_channels, out_channels]
# stride shape = the same order as the dimension specified with input
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

# tf.nn.max_pool
# value : 4-D tensor with [batch, in_height, in_width, in_channels]
# ksize : The size of the window for each dimension of the input tensor
# strides : The stride of the sliding window for each dimension of the input tensor
def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    # tf.reshape
    # If one component of shape is the special value -1, 
    # the size of that dimension is computed so that 
    # the total size remains constant.
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    # tf.nn.dropout(x, keep_prob, noise_shape=None, seed=None, name=None)
    # x : A tensor
    # keep_prob : A scalar Tensor with the same type as x
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [7]:
# Store layers weight & bias
weights = {
    # [filter_height, filter_width, in_channels, out_channels]
    # tf.random_normal : Outputs random values from a normal distribution
    #     input = shape, default mean = 0.0, default std = 1.0
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

In [26]:
# test
batch = 6
curridx = 8
a = np.array([[1,2], [3,4], [5,6], [7,8], [9,10], [1,2], [3,4], [5,6], [7,8], [9,10]])
b = np.concatenate((a[curridx:, :], a[0:batch-(10-curridx), :]), axis = 0)
print a.shape
print b.shape
print b

(10, 2)
(6, 2)
[[ 7  8]
 [ 9 10]
 [ 1  2]
 [ 3  4]
 [ 5  6]
 [ 7  8]]


In [29]:
# Launch the graph
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.6)
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    sess.run(init)
    step = 1
    start_idx = 0
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        #batch_x, batch_y = mnist.train.next_batch(batch_size)
        start_idx = start_idx % train_images.shape[0]
        if (start_idx + batch_size) < train_images.shape[0]:
            batch_x = train_images[start_idx:start_idx+batch_size, :]
            batch_y = train_labels[start_idx:start_idx+batch_size, :]
            start_idx += batch_size
        else:
            end_idx = batch_size - (train_images.shape[0] - start_idx)
            batch_x = np.concatenate((train_images[start_idx:, :], 
                                      train_images[0:end_idx, :]), axis = 0)
            batch_y = np.concatenate((train_labels[start_idx:, :], 
                                      train_labels[0:end_idx, :]), axis = 0)
            start_idx += batch_size
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"

    # Calculate accuracy for 256 mnist test images
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_images,
                                      y: test_labels,
                                      keep_prob: 1.})

Iter 1280, Minibatch Loss= 7337004.500000, Training Accuracy= 0.21875
Iter 2560, Minibatch Loss= 2079622.625000, Training Accuracy= 0.59375
Iter 3840, Minibatch Loss= 1203042.000000, Training Accuracy= 0.78125
Iter 5120, Minibatch Loss= 1101716.500000, Training Accuracy= 0.80469
Iter 6400, Minibatch Loss= 771436.250000, Training Accuracy= 0.79688
Iter 7680, Minibatch Loss= 703346.187500, Training Accuracy= 0.85938
Iter 8960, Minibatch Loss= 1360117.000000, Training Accuracy= 0.78906
Iter 10240, Minibatch Loss= 638400.000000, Training Accuracy= 0.85938
Iter 11520, Minibatch Loss= 409687.375000, Training Accuracy= 0.91406
Iter 12800, Minibatch Loss= 956221.875000, Training Accuracy= 0.82812
Iter 14080, Minibatch Loss= 1082338.750000, Training Accuracy= 0.82031
Iter 15360, Minibatch Loss= 489655.250000, Training Accuracy= 0.92188
Iter 16640, Minibatch Loss= 440682.125000, Training Accuracy= 0.92188
Iter 17920, Minibatch Loss= 377737.406250, Training Accuracy= 0.89062
Iter 19200, Minibatch